# Projekt: System rekomendacji muzyki

Celem projektu jest przetestowanie, jak dobrze osadzenia tworzone za pomocą BERTa mogą posłużyć za narzędzie do rekomendacji utworów muzycznych na bazie jakiegoś kontekstu, dokładniej playlisty użytkownika lub (krótkiej) historii słuchania.

Do projektu zostanie użyty tokenizator BERTa oraz API dla last.fm

In [2]:
import requests
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
from urllib.parse import quote
import time
from sklearn.metrics.pairwise import cosine_similarity
import json

In [3]:
try:
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    model = AutoModel.from_pretrained("bert-base-uncased")
    model.eval()
except:
    print("Błąd w inicjalizacji berta")


Poniższe funkcje pomogą w wyciągniu potrzebnych informacji z API. 

In [4]:

lastfm_headers = {
    "Authorization": "b0be60adc11d63391967e93883fc9e4d",
    "Content-Type": "application-json"
}

def fetch(url,headers):
    retries = 0
    
    while True:
        if retries >= 1000:
            print("Za dużo prób połączenia, terminuję")
            raise ValueError()
            
        
        response = requests.get(url,headers=headers)
        
        if response.status_code == 429:
            time.sleep(1)
            retries += 1
            continue
        
        if response.ok:
            return response.json()
        else:
            raise ValueError(f"Błąd żądania: {response.status_code} {response.text}")


lastfm_base_url= "http://ws.audioscrobbler.com/2.0/"

def get_track_info(artist, track):
    
    artist= quote(artist)
    track = quote(track)
    
    try:
        rsp = fetch(lastfm_base_url + f"?method=track.getInfo&artist={artist}&track={track}&api_key={lastfm_headers["Authorization"]}&format=json",lastfm_headers)
    except:
        return { 'track' : {}}
    return rsp

def get_track_tags(artist,track):
    artist= quote(artist)
    track = quote(track)
    
    try:
        rsp = fetch(lastfm_base_url + f"?method=track.gettoptags&artist={artist}&track={track}&api_key={lastfm_headers["Authorization"]}&format=json",lastfm_headers)
    except:
        return { 'toptags' : {'tag': []}}
    return rsp

def get_artist_info(artist):
    
    artist= quote(artist)
    
    try:
        rsp = fetch(lastfm_base_url + f"?method=artist.getinfo&artist={artist}&api_key={lastfm_headers["Authorization"]}&format=json",lastfm_headers)
    except:
        return { 'artist' : {}}
    return rsp

def get_top_tags():
    

    
    try:
        rsp = fetch(lastfm_base_url + f"?method=tag.getTopTags&api_key={lastfm_headers["Authorization"]}&format=json",lastfm_headers)
    except:
        return { 'toptags' : {'tag' : []}}
    return rsp


In [5]:
get_track_info("Anna Hamilton","Bad Liar")

{'track': {'name': 'Bad Liar',
  'mbid': '10442efd-8398-43e6-ab18-cc676b862506',
  'url': 'https://www.last.fm/music/Anna+Hamilton/_/Bad+Liar',
  'duration': '248000',
  'streamable': {'#text': '0', 'fulltrack': '0'},
  'listeners': '10612',
  'playcount': '57722',
  'artist': {'name': 'Anna Hamilton',
   'mbid': '9de7b385-709f-4c11-afb5-a38ca799fbf2',
   'url': 'https://www.last.fm/music/Anna+Hamilton'},
  'album': {'artist': 'Anna Hamilton',
   'title': 'Bad Liar - Single',
   'url': 'https://www.last.fm/music/Anna+Hamilton/Bad+Liar+-+Single',
   'image': [{'#text': '', 'size': 'small'},
    {'#text': '', 'size': 'medium'},
    {'#text': '', 'size': 'large'},
    {'#text': '', 'size': 'extralarge'}]},
  'toptags': {'tag': []}}}

In [6]:
get_track_tags("Anna Hamilton","Bad Liar")

{'toptags': {'tag': [],
  '@attr': {'artist': 'Anna Hamilton', 'track': 'Bad Liar'}}}

In [7]:
get_artist_info("Radiohead")

{'artist': {'name': 'Radiohead',
  'mbid': 'a74b1b7f-71a5-4011-9441-d0b5e4122711',
  'url': 'https://www.last.fm/music/Radiohead',
  'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'small'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'medium'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'large'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'extralarge'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'mega'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': ''}],
  'streamable': '0',
  'ontour': '0',
  'stats': {'listeners': '8098899', 'playcount': '1329342931'},
  'similar': {'artist': [{'name': 'Thom Yo

Poniżej pobierane są najpopularniejsze tagi w last.fm. Będą one służyły do oznaczania utworów muzycznych. Utwory będą mogły mieć jedynie te tagi. W szczególności niektóre utwory mogą nie mieć żadnego tagu, jeśli ich niepopularny tag nie należy do tego zbioru.

In [8]:
top_tags = [tag["name"] for tag in get_top_tags()["toptags"]["tag"]]
print(top_tags)

def embed_tags(tags):
    embedding = [1 if tag in tags else 0 for tag in top_tags]
    
    return embedding

def embed_count_tags(tags):
    embedding = []
    for tag_name in top_tags:
        flag = False
        for tag in tags:
            if tag['name'] == tag_name:
                embedding.append(tag['count'])
                flag = True
                break
            
        if not flag:
            embedding.append(0)
    return embedding

['rock', 'electronic', 'seen live', 'alternative', 'pop', 'indie', 'female vocalists', 'metal', 'alternative rock', 'jazz', 'classic rock', 'ambient', 'experimental', 'folk', 'indie rock', 'punk', 'Hip-Hop', 'hard rock', 'black metal', 'instrumental', 'singer-songwriter', 'dance', '80s', 'death metal', 'Progressive rock', 'heavy metal', 'hardcore', 'british', 'soul', 'chillout', 'electronica', 'rap', 'industrial', 'punk rock', 'Classical', 'Soundtrack', 'blues', 'thrash metal', '90s', 'metalcore', 'psychedelic', 'bookmark', 'acoustic', 'japanese', 'hip hop', 'post-rock', 'Progressive metal', 'House', 'german', 'techno']


In [9]:
def bert_features(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_emb = outputs.last_hidden_state[:, 0, :].squeeze(0)
    cls_emb = cls_emb[:min(300,len(cls_emb))]
    return cls_emb.cpu().numpy()

In [10]:
def fetch_wikipedia_intro(artist_name):
    artist_enc = quote(artist_name)
    url = f"https://en.wikipedia.org/w/api.php?action=query&prop=extracts&exintro=1&explaintext=1&format=json&titles={artist_enc}"
    
    response = requests.get(url)
    data = response.json()
    
    pages = data.get("query", {}).get("pages", {})
    if not pages:
        return None
    
    page = next(iter(pages.values()))
    extract = page.get("extract", "")
    return extract

bio_cache = {}
def embed_bio(artist_info):
    artist= artist_info.get("artist").get('name')
    
    if artist in bio_cache:
        return bio_cache[artist]

    summary = artist_info['artist']['bio']['summary']
    
    if len(summary) > 0:
        e = bert_features(summary)
    else:
        fallback = fetch_wikipedia_intro(artist)
        
        if fallback:
            e = bert_features(fallback)
        else:
            tags = artist_info.get("artist", {}).get("tags", {}).get("tag", [])
            tag_text = " ".join([t["name"] for t in tags]) if tags else ""
            fallback_text = f"{artist} {tag_text}".strip()
            
            e = bert_features(fallback_text) if fallback_text else bert_features(artist)
        
    bio_cache[artist] = e
            
    return e

In [11]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm


# W jaki sposób będą tworzone osadzenia?

Osadzenia będą miały 3 segmenty.

1. Segment pierwszy - osadzenie opisu autora. Każdy autor będzie miał swój krótki opis. Ten opis będzie potem podlegał osadzeniu przez BERTa.
2. Segement drugi - osadzenie onehot dla tagów artysty
3. Segment trzeci - osadzenie onehot dla tagów piosenki

Poniżej mogą być widoczne pozostałości po czwartym segmencie, czyli roku wydania, ale ten segemnt został usunięty, ponieważ okazało się, że bardzo dużo utworów w last.fm. nie posiada roku wydania.

In [12]:
def create_embedding(artist, track):
    track_tags = get_track_tags(artist,track)
    track_info = get_track_info(artist,track)
    artist_info = get_artist_info(artist)
    
    bio_embedding = embed_bio(artist_info)
    artist_tags_embedding = embed_tags([d['name'] for d in artist_info['artist']['tags']['tag']])
    track_embedding = embed_count_tags(track_tags['toptags']['tag'])
    
    try:
        release_year = int(track_info.get("track").get("wiki").get("published").split(" ")[2][:4])
    except:
        release_year = 0

    return (
        normalize(bio_embedding),
        normalize(artist_tags_embedding),
        normalize(track_embedding),
        (release_year-1970)/100
    )


In [13]:
create_embedding("Anna Hamilton","Bad Liar")

(array([-1.59459338e-02, -4.89350362e-03, -2.10872088e-02, -6.94733858e-02,
        -5.94227351e-02, -4.67963628e-02,  5.08333221e-02,  8.39311704e-02,
         2.77800970e-02, -2.99097393e-02, -4.43666242e-02, -2.65559386e-02,
         2.43179537e-02,  3.32399160e-02,  1.02949686e-01, -2.46328525e-02,
        -3.50924879e-02,  1.10920966e-01,  8.50345287e-03, -8.25321767e-04,
         1.03559000e-02, -4.73584831e-02,  2.59153098e-02, -3.82981338e-02,
         2.19471753e-02, -1.88665874e-02, -1.63168628e-02, -5.73364496e-02,
        -2.47756634e-02, -3.85840163e-02, -3.62892225e-02,  1.05300294e-02,
        -2.29989667e-03, -3.17202099e-02,  5.57763949e-02, -4.36972156e-02,
        -1.89377610e-02,  3.40378797e-03,  4.80241627e-02,  4.63149287e-02,
        -7.57345557e-02, -9.38084796e-02,  8.61778557e-02,  7.18816463e-03,
         9.00418684e-03, -1.82395242e-02, -3.88154209e-01,  3.70298699e-02,
        -1.19101340e-02, -2.46936623e-02,  6.28232658e-02, -1.87374335e-02,
         8.6

# Jak porównywane są osadzenia?
Zanurzenia będą porównywane przez podobieństwo cosinusowe każdego segmentu osobno, z pewnymi wagami dla każdego segmentu. Na tą chwilę, osadzenia onehot dla tagów są dwa razy ważniejsze niż osadzenie tekstowe opisu artysty.

In [14]:
def compare_embeddings(e1,e2):
    
    sim = (
        0.2 * cosine_similarity(np.array(e1[0]).reshape(1, -1),
                                np.array(e2[0]).reshape(1, -1))[0, 0] +
        0.4 * cosine_similarity(np.array(e1[1]).reshape(1, -1),
                                np.array(e2[1]).reshape(1, -1))[0, 0] +
        0.4 * cosine_similarity(np.array(e1[2]).reshape(1, -1),
                                np.array(e2[2]).reshape(1, -1))[0, 0]
    )
    return sim

# Baza

Istotnym elementem jest baza osadzeń, w której będzie można znajdować najbardziej podobne wektory. 
Zdecydowałem się użyć bazy o wielkości 50 tysięcy utworów. Dlaczego 50 tysięcy? Być może dlatego, że ta liczba wydawała mi się pewnym balansem między wielkością, która miała przynieść sensowne wyniki, a optymalnym czasem obliczeń. Osadzanie utworów trwało w tym wypadku ok. 18 godzin. Być może użycie większej ilości utworów dałoby lepsze wyniki.

Użyty baza utworów (ok. 50K z nich):
https://www.kaggle.com/datasets/priyamchoksi/spotify-dataset-114k-songs

In [15]:

def train(data, filename="emb.json"):
    results = {}


    try:
        with open(filename, "r", encoding="utf-8") as f:
            results = json.load(f)
    except FileNotFoundError:
        pass  

    with open("emb.txt","w",encoding="utf-8") as txt:
        

        for _, row in tqdm(data.iterrows()):
            track = row["track_name"]
            artist = row["artists"].split(";")[0]
            key = f"{track} - {artist}"



            try:
                emb = create_embedding(artist, track)
            
                emb_lists = [e.tolist() if isinstance(e, np.ndarray) else e for e in emb]
                results[key] = emb_lists

            except Exception as e:
                print(f"Błąd przy {track} - {artist}: {e}")
                continue
            
            text = ""
            text += track + "|" + artist + "|"
            text += ",".join(f"{x:.6f}" for x in emb[0]) + "|"
            text += ",".join(f"{x:.6f}" for x in emb[1]) + "|"
            text += ",".join(f"{x:.6f}" for x in emb[2]) + "|"
            text += str(emb[3]) + "\n"
            txt.write(str(text))


    
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

In [16]:
data = pd.read_csv("dataset.csv")
data[["artists","track_name"]]


,artists,track_name
0,Gen Hoshino,Comedy
1,Ben Woodward,Ghost - Acoustic
2,Ingrid Michaelson;ZAYN,To Begin Again
3,Kina Grannis,Can't Help Falling In Love
4,Chord Overstreet,Hold On
...,...,...
113995,Rainy Lullaby,Sleep My Little Boy
113996,Rainy Lullaby,Water Into Light
113997,Cesária Evora,Miss Perfumado
113998,Michael W. Smith,Friends


In [17]:
def txt_to_dict(filename="emb.txt"):
    
    results = {}
    with open(filename,"r",encoding="utf-8") as f:
        data = f.read().split("\n")
        
        for line in data:
            a = line.split("|")
            print(a)
            track = a[0]
            artist = a[1]
            
            bio_emb = [float(x) for x in a[2].split(",")]
            artist_emb = [float(x) for x in a[3].split(",")]
            track_emb = [float(x) for x in a[4].split(",")]
            
            results[f"{track} - {artist}"] = [bio_emb,artist_emb,track_emb]
            
        
    return results
        

In [18]:
def json_to_dict(filename="emb.json"):
    with open(filename, "r", encoding="utf-8") as f:
        return json.load(f)

In [19]:
#train()


# res = txt_to_dict()

# with open("emb.json", "w", encoding="utf-8") as f:
#     json.dump(res, f, ensure_ascii=False)

# Osadzenia playlist

Docelowo chciałem, aby rekomendacje były na bazie playlist / zbiorów utworów. Wymagało to utworzenia jakiegoś sposobu osadzania playlist. Użyłem dosyć prostego sposobu, czyli średniej na każdym polu osadzenia każdej piosenki.

In [20]:

def embed_playlist(playlist):

    embeddings = [create_embedding(song['artist'], song['track']) for song in playlist]
    

    mean_parts = []
    for part_group in zip(*embeddings):

        stacked = np.vstack(part_group) 
        mean_vec = stacked.mean(axis=0)
        mean_parts.append(np.array(mean_vec))
    
    return tuple(mean_parts)


# Sposoby wyszukiwania najlepszych wektorów


- Bruteforce

Jakimś sposobem (jak się okaże, całkiem niezłym jakościowo) jest porównanie każdego osadzenia w bazie z osadzeniem playlisty i posortowanie po wynikach. Działa jednak dosyć wolno. Dla bazy o wielkości 50tys. jest to dopuszczalna długość, ale bardzo źle skaluje się dla większych baz.

In [21]:
def top_n_bruteforce(data, playlist_emb,N=10):
    
    return sorted(data.keys(),key= lambda k: compare_embeddings(data[k],playlist_emb),reverse=True)[:N]

In [22]:
pop_playlist = [
    {'track': 'Blinding Lights', 'artist': 'The Weeknd'},
    {'track': 'As It Was', 'artist': 'Harry Styles'},
    {'track': 'Levitating', 'artist': 'Dua Lipa'},
    {'track': 'Bad Habits', 'artist': 'Ed Sheeran'},
    {'track': 'Stay', 'artist': 'The Kid LAROI'},
    {'track': 'Peaches', 'artist': 'Justin Bieber'},
    {'track': 'Watermelon Sugar', 'artist': 'Harry Styles'},
    {'track': 'Save Your Tears', 'artist': 'The Weeknd'},
    {'track': 'drivers license', 'artist': 'Olivia Rodrigo'},
    {'track': 'good 4 u', 'artist': 'Olivia Rodrigo'},
]

rock_playlist = [
    {'track': 'Bohemian Rhapsody', 'artist': 'Queen'},
    {'track': 'Stairway to Heaven', 'artist': 'Led Zeppelin'},
    {'track': 'Hotel California', 'artist': 'Eagles'},
    {'track': 'Sweet Child O\' Mine', 'artist': 'Guns N\' Roses'},
    {'track': 'Smoke on the Water', 'artist': 'Deep Purple'},
    {'track': 'Livin\' on a Prayer', 'artist': 'Bon Jovi'},
    {'track': 'Back in Black', 'artist': 'AC/DC'},
    {'track': 'Paint It, Black', 'artist': 'The Rolling Stones'},
    {'track': 'November Rain', 'artist': 'Guns N\' Roses'},
    {'track': 'Highway to Hell', 'artist': 'AC/DC'},
]


hiphop_playlist = [
    {'track': 'Sicko Mode', 'artist': 'Travis Scott'},
    {'track': 'God\'s Plan', 'artist': 'Drake'},
    {'track': 'HUMBLE.', 'artist': 'Kendrick Lamar'},
    {'track': 'Lose Yourself', 'artist': 'Eminem'},
    {'track': 'Old Town Road', 'artist': 'Lil Nas X'},
    {'track': 'Juicy', 'artist': 'The Notorious B.I.G.'},
    {'track': 'Industry Baby', 'artist': 'Lil Nas X'},
    {'track': 'Life Goes On', 'artist': 'Lil Baby'},
    {'track': 'Way 2 Sexy', 'artist': 'Drake'},
    {'track': 'Alright', 'artist': 'Kendrick Lamar'},
]

classical_playlist = [
    {'track': 'Für Elise', 'artist': 'Ludwig van Beethoven'},
    {'track': 'Canon in D', 'artist': 'Johann Pachelbel'},
    {'track': 'The Four Seasons: Spring', 'artist': 'Antonio Vivaldi'},
    {'track': 'Swan Lake', 'artist': 'Pyotr Ilyich Tchaikovsky'},
    {'track': 'Clair de Lune', 'artist': 'Claude Debussy'},
    {'track': 'Moonlight Sonata', 'artist': 'Ludwig van Beethoven'},
    {'track': 'Requiem: Lacrimosa', 'artist': 'Wolfgang Amadeus Mozart'},
    {'track': 'Symphony No.5', 'artist': 'Ludwig van Beethoven'},
    {'track': 'Ride of the Valkyries', 'artist': 'Richard Wagner'},
    {'track': 'Adagio for Strings', 'artist': 'Samuel Barber'},
]





In [23]:
data = json_to_dict()

In [24]:
pop_emb = embed_playlist(pop_playlist)
rock_emb = embed_playlist(rock_playlist)
classical_emb = embed_playlist(classical_playlist)
hiphop_emb = embed_playlist(hiphop_playlist)

In [25]:
top_pop = top_n_bruteforce(data,pop_emb,N=50)
top_rock = top_n_bruteforce(data,rock_emb,N=50)
top_classical = top_n_bruteforce(data,classical_emb,N=50)
top_hiphop = top_n_bruteforce(data,hiphop_emb,N=50)

In [26]:
top_pop

['2002 - Anne-Marie',
 'Me And My Broken Heart - Rixton',
 'Levitating (feat. DaBaby) - Dua Lipa',
 'Levitating - Dua Lipa',
 "Don't Start Now - Dua Lipa",
 'Good Ones - Charli XCX',
 'Break My Heart - Dua Lipa',
 'Sucker - Jonas Brothers',
 'IDGAF - Dua Lipa',
 'Tu Falta De Querer - Mon Laferte',
 "I'm a Mess - Bebe Rexha",
 'I Don’t Wanna Live Forever (Fifty Shades Darker) - ZAYN',
 'Nothing Breaks Like a Heart (feat. Miley Cyrus) - Mark Ronson',
 'Uptown Funk (feat. Bruno Mars) - Mark Ronson',
 'Physical - Dua Lipa',
 'Bubblegum Bitch - MARINA',
 "Hips Don't Lie (feat. Wyclef Jean) - Shakira",
 'Kings & Queens - Ava Max',
 'positions - Ariana Grande',
 'Stuck with U (with Justin Bieber) - Ariana Grande',
 'thank u, next - Ariana Grande',
 'boyfriend (with Social House) - Ariana Grande',
 'God is a woman - Ariana Grande',
 'Side To Side - Ariana Grande',
 'Love Me Harder - Ariana Grande',
 'Sorry Not Sorry - Demi Lovato',
 'Sweet but Psycho - Ava Max',
 'Kiss and Make Up - Dua Lipa',

In [27]:
top_rock

["Sweet Child O' Mine - Guns N' Roses",
 "Knockin' On Heaven's Door - Guns N' Roses",
 'Walk This Way - Aerosmith',
 "Paradise City - Guns N' Roses",
 "November Rain - Guns N' Roses",
 'Sweet Emotion - Aerosmith',
 "Welcome To The Jungle - Guns N' Roses",
 'Poison - Alice Cooper',
 'Dream On - Aerosmith',
 'Good Times Bad Times - 1993 Remaster - Led Zeppelin',
 "Don't Cry (Original) - Guns N' Roses",
 'Still Loving You - Scorpions',
 'Wind Of Change - Scorpions',
 'Wind of Change - Scorpions',
 'You Shook Me All Night Long - AC/DC',
 'Thunderstruck - AC/DC',
 'T.N.T. - AC/DC',
 'Back In Black - AC/DC',
 'Hells Bells - AC/DC',
 'Dirty Deeds Done Dirt Cheap - AC/DC',
 'Highway to Hell - AC/DC',
 'Black Betty - Ram Jam',
 'The Final Countdown - Europe',
 "(Don't Fear) The Reaper - Blue Öyster Cult",
 'Panama - 2015 Remaster - Van Halen',
 'More Than a Feeling - Boston',
 'Black Dog - Remaster - Led Zeppelin',
 "I'd Love to Change the World - 2004 Remaster - Ten Years After",
 'Simple Man 

In [28]:
top_classical

['Piano Sonata No. 14 in C-Sharp Minor, Op. 27 No. 2 "Moonlight": I. Adagio sostenuto - Ludwig van Beethoven',
 'Symphony No. 6 in F Major, Op. 68 "Pastoral": III. Lustiges Zusammensein der Landleute. Allegro - Ludwig van Beethoven',
 'Bagatelle in A Minor, WoO 59 "Für Elise" - Ludwig van Beethoven',
 'Symphony No. 8 in F Major, Op. 93: I. Allegro vivace e con brio (Live) - Ludwig van Beethoven',
 'Violin Concerto in D Major, Op. 61: II. Larghetto - Ludwig van Beethoven',
 'Piano Concerto No. 4 in G Major, Op. 58: III. Rondo. Vivace - Ludwig van Beethoven',
 'Triple Concerto in C Major, Op. 56: III. Rondo alla Polacca - Live at Philharmonie, Berlin / 2019 - Ludwig van Beethoven',
 'Sonata No. 14 "Moonlight" in C-Sharp Minor", Op. 27 No. 2: I. Adagio sostenuto - Ludwig van Beethoven',
 'Für Elise, WoO 59 - Ludwig van Beethoven',
 'Bagatelle No. 25 in A Minor, WoO 59 "Für Elise" - Ludwig van Beethoven',
 'Moonlight Sonata (First Movement from Piano Sonata No. 14, Op. 27 No. 2) - Ludwig v

In [29]:
top_hiphop

['Please Me - Cardi B',
 'I Like It - Cardi B',
 'Forgot About Dre - Dr. Dre',
 'The Next Episode - Dr. Dre',
 "What's The Difference - Dr. Dre",
 'Juicy - 2005 Remaster - The Notorious B.I.G.',
 'BILLY - 6ix9ine',
 'GUMMO - 6ix9ine',
 'KEKE - 6ix9ine',
 'Keep Ya Head Up - 2Pac',
 "Drop It Like It's Hot - Snoop Dogg",
 'Still D.R.E. - Dr. Dre',
 'Changes - 2Pac',
 'It Was A Good Day - Ice Cube',
 'Ambitionz Az A Ridah - 2Pac',
 'Super Freaky Girl - Nicki Minaj',
 'Do For Love - 2Pac',
 'Vegas (From the Original Motion Picture Soundtrack ELVIS) - Doja Cat',
 'N.Y. State of Mind - Nas',
 "Life's a Bitch (feat. AZ & Olu Dara) - Nas",
 'Regulate - Warren G',
 '16 Lines - Lil Peep',
 'You Know How We Do It - Ice Cube',
 'E. Coli (feat. Earl Sweatshirt) - The Alchemist',
 'Broken - Lund',
 'INDUSTRY BABY (feat. Jack Harlow) - Lil Nas X',
 'Jocelyn Flores - XXXTENTACION',
 'Riot - XXXTENTACION',
 'the remedy for a broken heart (why am I so in love) - XXXTENTACION',
 'changes - XXXTENTACION',


Dla każdego rodzaju muzyki wynik wydaje się być sensowny. Z początku obawiałem się, że wynikiem będą jedynie piosenki tego samego artysty (przez to że osadzenie utworu korzysta w sporej mierze danych artysty), ale jest też wiele więcej i niektóre piosenki innych artystów mają nawet lepsze wyniki, więc jest to jakoś zróżnicowane.

Co ciekawego można byłoby sprawdzić lub dodać:

- KMeans clustering i wyszukiwanie wg jakiegoś kryterium, ale tylko po sąsiadach z klastra
- grafowa baza - wymaga totalnie innego podejścia, ale jest ciekawa
- KNN
- model, gdzie tokenem będzie utwór - na bazie sekwencji utworów przewidywałby kolejny token-utwór i na tej bazie tworzył rekomendacje (wymagałoby to jednak dużo uczenia oraz dużych korpusów)